In [1]:
import datetime as dt
import difflib
import json
import math
import os
import string
import sys
from datetime import timedelta
from random import sample

import folium
import matplotlib as mpl
import matplotlib.pyplot as plt
import missingno as msno
import numpy as np
import pandas as pd
import pyreadstat
import seaborn as sns
import statsmodels.tools.tools as sm
import statsmodels.base as sb
from folium.features import CustomIcon
from folium.plugins import FastMarkerCluster, HeatMap, MarkerCluster
from linearmodels.panel.model import PooledOLS
from scipy.signal import savgol_filter
from sklearn.decomposition import PCA
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from statsmodels.stats.stattools import durbin_watson
from statsmodels.tools.eval_measures import aic, rmse
from statsmodels.tsa.api import VAR
from statsmodels.tsa.stattools import adfuller
import yfinance as yf
from yahoofinancials import YahooFinancials

import pickle

In [2]:
postcodecsv = pd.read_csv("au_postcodes.csv")
# References:
# https://blog.greglow.com/2019/11/05/power-bi-creating-a-topojson-file-of-australian-postcodes-for-use-with-shape-map/
# https://mapshaper.org/
# https://www.abs.gov.au/AUSSTATS/abs@.nsf/DetailsPage/1270.0.55.003July%202016?OpenDocument

from statistics import mean, median

import pandas as pd
from scipy import stats

ndf = pd.read_json("POA_2016_AUST.json")
ndf1 = ndf["features"].apply(pd.Series)
ndf2 = ndf1["geometry"].apply(pd.Series)
ndf3 = ndf1["properties"].apply(pd.Series)
border = ndf2["coordinates"]

depth = lambda L: isinstance(L, list) and max(map(depth, L)) + 1


def flatten(d, l=1):
    for i in d:
        yield from ([i] if l == 1 else flatten(i, l - 1))


def centerlocation(index):
    tmplst = list(flatten(border[index], l=depth(border[index])))
    longi = median([i for i in tmplst if i > 0])
    lati = median([i for i in tmplst if i < 0])
    return [longi, lati]


postcodedf = []
i = 0
for i in range(len(ndf)):
    if depth(border[i]) > 0:
        tpd = {
            "postcode": [ndf3["POA_CODE16"].iloc[i]],
            "longitude": [centerlocation(i)[0]],
            "latitude": [centerlocation(i)[1]],
        }
        tpdf = pd.DataFrame(tpd)
        postcodedf.append(tpdf)
    else:
        print(i)
        continue
    i = i + 1
postcodedf = pd.concat(postcodedf, ignore_index=True)

2668
2669


In [3]:
data = pd.read_csv(
    "./markettrends0.csv",
    dtype={
        "state": "str",
        "sa3_name16": "str",
        "sa4_name16": "str",
        "postcode": "str",
        "state": "str",
        "property_type": "str",
    },
)
ndata = data.fillna(
    {"Volume of new rental listings (1 month)": 0, "Volume of sales (1 month)": 0}
).dropna(subset=["postcode"])
ndatahouses = ndata[:][ndata.property_type == "Houses"]
ndataunits = ndata[:][ndata.property_type == "Units"]

In [4]:
# ndatahouses

In [5]:
# ndatahouses['postcode'].loc[ndatahouses['sa3_name16'] == 'Darwin City' ].unique()


In [6]:
# So far, Houses only
UniqueNames_sa3 = ndatahouses.sa3_name16.unique()[:-1]


In [7]:
# DataFrameDict_sa3 
# DataFrameDict_sa3 = {elem: pd.DataFrame for elem in UniqueNames_sa3}
# for key in DataFrameDict_sa3.keys():
#     DataFrameDict_sa3[key] = ndata[:][ndata.sa3_name16 == key]

In [8]:
year = list(range(2001, 2021))

In [8]:
DataFrameDict_HILDAp = {elem: pd.DataFrame for elem in year}
for key in DataFrameDict_HILDAp.keys():
    order = key - 2001
    ini = string.ascii_lowercase[order]
    df, meta = pyreadstat.read_dta(f"Rperson_{ini}200u.dta")
    df.columns = df.columns.str[1:]
    DataFrameDict_HILDAp[key] = df

In [9]:
# Pickle1 
# with open('DataFrameDict_HILDAp.pickle', 'wb') as f:
#     pickle.dump(DataFrameDict_HILDAp, f)

In [9]:
DataFrameDict_HILDA_cw = {elem: pd.DataFrame for elem in year}
for key in DataFrameDict_HILDA_cw.keys():
    order = key - 2001
    ini = string.ascii_lowercase[order]
    df, meta = pyreadstat.read_dta(f"Combined_{ini}200u.dta")
    df.columns = df.columns.str[1:]
    DataFrameDict_HILDA_cw[key] = df

In [ ]:
with open('DataFrameDict_HILDA_cw.pickle', 'wb') as f:
    pickle.dump(DataFrameDict_HILDA_cw, f)

In [ ]:
DataFrameDict_HILDA = {elem: pd.DataFrame for elem in year}
for key in DataFrameDict_HILDA.keys():
    order = key - 2001
    ini = string.ascii_lowercase[order]
    df, meta = pyreadstat.read_dta(f"Household_{ini}200u.dta")
    df.columns = df.columns.str[1:]
    DataFrameDict_HILDA[key] = df




In [ ]:
with open('DataFrameDict_HILDA.pickle', 'wb') as f:
    pickle.dump(DataFrameDict_HILDA, f)

In [ ]:

list_postcodes = DataFrameDict_HILDA[2001]["hhpcode"]
for key in DataFrameDict_HILDA.keys():
    list_ny = DataFrameDict_HILDA[key]["hhpcode"]
    list_postcodes = list(set(list_postcodes).intersection(list_ny))
count_dict = dict()
for key in DataFrameDict_HILDA.keys():
    for item in DataFrameDict_HILDA[key]["hhpcode"].unique():
        if item in count_dict:
            count_dict[item] += 1
        else:
            count_dict[item] = 1
df_allpostcode = pd.DataFrame(count_dict.items())

In [ ]:
HILDAallpc = df_allpostcode[0].values

In [ ]:
df_retainedpc = df_allpostcode.drop(df_allpostcode[df_allpostcode[1] < 20].index)

In [ ]:
UniquePostcode = df_retainedpc[0]
DataFrameDict_postcode = {elem: pd.DataFrame for elem in UniquePostcode}
for key in DataFrameDict_postcode.keys():
    DataFrameDict_postcode[key] = ndatahouses[:][ndatahouses.postcode == key]

In [ ]:
def unique_non_null(s):
    return s.dropna().unique()


# placepc  = postcodedf['Postcode']

growthrate = []

for key in DataFrameDict_postcode.keys():
    if sum(postcodedf["postcode"] == key) > 0:
        # ind = placepc.loc[placepc == pc].index[0]
        ind = postcodedf.loc[postcodedf["postcode"] == key].index[0]
        df = DataFrameDict_postcode[key]
        if len(df["Hedonic Home Value Index"]) > 360:
            tpd = {
                "UniqueNames_sa3": DataFrameDict_postcode[key]["sa3_name16"].unique(),
                "S/T": DataFrameDict_postcode[key]["state"].unique(),
                #'Postcode':[postcodedf.loc[ind,'Postcode']],
                # 'longitude':[postcodedf.loc[ind,'longitude']],
                #'latitude':[postcodedf.loc[ind,'latitude']],
                "Postcode": [postcodedf.loc[ind, "postcode"]],
                "longitude": [postcodedf.loc[ind, "longitude"]],
                "latitude": [postcodedf.loc[ind, "latitude"]],
                "Growth_Houses": [
                    df["Hedonic Home Value Index"].iloc[-1]
                    / df["Hedonic Home Value Index"].iloc[359]
                ],
                "FinalHPI_Houses": [df["Hedonic Home Value Index"].iloc[-1]],
                # 'Suburbs':", ". join( ((postcodecsv.loc[(postcodecsv['postcode'] ==int(postcodedf.loc[ind,'Postcode']))]['place_name']).tolist()))}
                "Suburbs": ", ".join(
                    (
                        (
                            postcodecsv.loc[(postcodecsv["postcode"] == int(key))][
                                "place_name"
                            ]
                        ).tolist()
                    )
                ),
            }
            tpdf = pd.DataFrame(tpd)
            growthrate.append(tpdf)
    else:
        print("area(value) %s: data incomplete" % (key))
        continue

growthrate = pd.concat(growthrate, ignore_index=True)
growthrate["Postcode"] = growthrate["Postcode"].astype(str)

In [ ]:
ndfsa3 = pd.read_json("SA3_2016_AUST.json")
ndf1sa3 = ndfsa3["features"].apply(pd.Series)
ndf2sa3 = ndf1sa3["geometry"].apply(pd.Series)
ndf3sa3 = ndf1sa3["properties"].apply(pd.Series)
borderdsa3 = ndf2sa3["coordinates"]
placename = ndf3sa3["SA3_NAME16"]

In [ ]:



growthsa3 = []

i = -1

for key in DataFrameDict_sa3.keys():
    i = i + 1
    ind = placename[placename == difflib.get_close_matches(key, placename)[0]].index[0]
    df = DataFrameDict_sa3[key]
    tpd = {
        "UniqueNames_sa3": [DataFrameDict_sa3[key]["sa3_name16"].unique()[0]],
        "sa3code": [ndf3sa3.loc[ind, "SA3_CODE16"]],
        "FinalHPI_Houses": [
            DataFrameDict_sa3[key][df.value_at_date == "2021-05-31"][
                "Hedonic Home Value Index"
            ].mean()
        ],
    }
    tpdf = pd.DataFrame(tpd)
    growthsa3.append(tpdf)
growthsa3 = pd.concat(growthsa3, ignore_index=True)

# New Map Updated

- Suburbs without enough samples in HILDA are removed. 
- Standard of this version: missing rate is greater than 0% or suburbs not included in HILDA data 
- Link: https://lyulingfeng95.github.io/Project1_LF/


In [ ]:
folium_map4 = folium.Map(location=[-30, 138], zoom_start=4.4, titles="cartodbpositron")


f = open("POA_2016_AUST.json")
postcode_geo = json.load(f)
postcodedata2 = pd.DataFrame(
    {"postcode": df_retainedpc[0], "fill": [1] * len(df_retainedpc[0])}
)

folium.Choropleth(
    geo_data=postcode_geo,
    name="choropleth",
    data=postcodedata2,
    fill_color="BrBG",
    columns=["postcode", "fill"],
    key_on="feature.properties.POA_CODE16",
    fill_opacity=0.4,
    line_opacity=0.6,
).add_to(folium_map4)

mc1 = MarkerCluster(name="Marker Cluster")
df = growthrate
for index, row in df.iterrows():
    html = " Postcode: {}<br><br>Suburb(s): {}<br><br> Location (SA3): {} <br><br>State: {} <br><br> Fixed Based Growth Rate since 31/12/2009: {}  ".format(
        row["Postcode"],
        row["Suburbs"],
        row["UniqueNames_sa3"],
        row["S/T"],
        format(row["Growth_Houses"], ".4f"),
    )

    iframe = folium.IFrame(html)
    popup = folium.Popup(iframe, min_width=400, max_width=400)
    folium.CircleMarker(
        location=[row["latitude"], row["longitude"]],
        color="blue",
        fill_opacity=math.pow(row["Growth_Houses"], 4) / 10,
        weight=0.8,
        popup=popup,
        fill=True,
    ).add_to(mc1)

mc1.add_to(folium_map4)

folium.LayerControl().add_to(folium_map4)

folium_map4

In [ ]:
# folium_map4.save("index.html")

In [ ]:
# The average house price index growth rate.
aveHPI = []
lst = ["postcode", "date", "logHPI", "logHPIdiff"]
# Calling DataFrame constructor on list
for key in DataFrameDict_postcode.keys():
    df = pd.DataFrame([], columns=lst)
    df["Description"] = DataFrameDict_postcode[key]["value_at_date"].loc[
        ("2019-12" > DataFrameDict_postcode[key]["value_at_date"])
        & (DataFrameDict_postcode[key]["value_at_date"] > "1990-12")
    ]
    df["logHPI"] = np.log2(
        DataFrameDict_postcode[key]["Hedonic Home Value Index"].loc[
            ("2019-12" > DataFrameDict_postcode[key]["value_at_date"])
            & (DataFrameDict_postcode[key]["value_at_date"] > "1990-12")
        ]
    )
    df["postcode"] = key
    df["logHPIdiff"] = df["logHPI"].diff(1)
    aveHPI.append(df)

aveHPIdf = pd.concat(aveHPI)
aveHPIdf = aveHPIdf.reset_index(drop=True)
agg = aveHPIdf.groupby("Description")["logHPIdiff"].agg(["mean"]).reset_index()
agg.dropna()

In [ ]:
xls1 = pd.read_excel("f01hist.xls", sheet_name="Data", header=2)
xls1 = xls1.drop(xls1.index[0:8], axis=0)
xls1["Description"] = pd.to_datetime(xls1["Description"])
xls1["Description"] = xls1["Description"].dt.date.apply(lambda x: x.strftime("%Y-%m"))
xls3 = pd.read_excel("f11hist-1969-2009.xls", sheet_name="Data", header=2)
xls3 = xls3.drop(xls3.index[0:8], axis=0)
xls3["Description"] = pd.to_datetime(xls3["Description"])
xls3["Description"] = xls3["Description"].dt.date.apply(lambda x: x.strftime("%Y-%m"))
xls4 = pd.read_excel("f11hist.xls", sheet_name="Data", header=2)
xls4 = xls4.drop(xls4.index[0:8], axis=0)
xls4["Description"] = pd.to_datetime(xls4["Description"])
xls4["Description"] = xls4["Description"].dt.date.apply(lambda x: x.strftime("%Y-%m"))
xls34 = pd.concat([xls3, xls4], axis=0)
xls5 = pd.read_excel("g01hist.xls", sheet_name="Data", header=2)
xls5 = xls5.drop(xls5.index[0:8], axis=0)
xls5["Description"] = pd.to_datetime(xls5["Description"])
xls5["Description"] = xls5["Description"].dt.date.apply(lambda x: x.strftime("%Y-%m"))
xls6 = pd.read_excel("h01hist.xls", sheet_name="Data", header=2)
xls6 = xls6.drop(xls6.index[0:8], axis=0)
xls6["Description"] = pd.to_datetime(xls6["Description"])
xls6["Description"] = xls6["Description"].dt.date.apply(lambda x: x.strftime("%Y-%m"))
xls7 = pd.read_excel("h03hist.xls", sheet_name="Data", header=2)
xls7 = xls7.drop(xls7.index[0:8], axis=0)
xls7["Description"] = pd.to_datetime(xls7["Description"])
xls7["Description"] = xls7["Description"].dt.date.apply(lambda x: x.strftime("%Y-%m"))
# Download ASX data from Yahoo Finance

ASX200data = yf.download('^AXJO', start='1999-01-01', end='2021-06-01', interval = "1mo", progress=False)
ASX200data = ASX200data.reset_index()
ASX200data["Description"] = pd.to_datetime(ASX200data["Date"])
ASX200data["Description"] = ASX200data["Description"].dt.date.apply(lambda x: x.strftime("%Y-%m"))

In [ ]:
# macvar = pd.DataFrame().assign(
#     Description=xls7.loc[
#         ("2021-06" > xls7["Description"]) & (xls7["Description"] > "1998-12")
#     ]["Description"]
# )
# macvar.reset_index(drop=True, inplace=True)

# macvar = pd.merge(
#     macvar,
#     xls1[["Description", "Cash Rate Target; monthly average"]],
#     on="Description",
#     how="left",
# )
# macvar = pd.merge(
#     macvar,
#     xls34[["Description", "AUD/USD Exchange Rate; see notes for further detail."]],
#     on="Description",
#     how="left",
# )
# macvar = pd.merge(
#     macvar,
#     xls5[["Description", "Consumer price index; All groups"]],
#     on="Description",
#     how="left",
# )
# macvar = pd.merge(
#     macvar,
#     xls6[["Description", "Gross domestic product (GDP); Chain volume"]],
#     on="Description",
#     how="left",
# )
# macvar = pd.merge(
#     macvar,
#     xls7[["Description", "Retail sales; All industries; Current price"]],
#     on="Description",
#     how="left",
# )
# macvar = pd.merge(
#     macvar,
#     xls7[["Description", "Private dwelling approvals"]],
#     on="Description",
#     how="left",
# )

# macvar = macvar.set_axis(
#     ["date", "ir", "exr", "cpi", "gdp", "rs", "pda"], axis=1, inplace=False
# )
# macvar["ir"] = macvar["ir"].astype(float, errors="raise")
# macvar["exr"] = macvar["exr"].astype(float, errors="raise")
# macvar["cpi"] = macvar["cpi"].astype(float, errors="raise")
# macvar["gdp"] = macvar["gdp"].astype(float, errors="raise")
# macvar["rs"] = macvar["rs"].astype(float, errors="raise")
# macvar["pda"] = macvar["pda"].astype(float, errors="raise")
# macvar = macvar.interpolate()
# macvarst = macvar


# lag = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]
# threshold = 0.01
# for i in lag:
#     if adfuller(macvar["ir"].pct_change(i).dropna(how="all"))[1] < threshold:
#         macvarst[f"ir{i}"] = macvar["ir"].pct_change(i)
#     if adfuller(macvar["ir"].pct_change(i).diff().dropna(how="all"))[1] < threshold:
#         macvarst[f"ir{i}_2"] = macvar["ir"].pct_change(i).diff()

#     if adfuller(macvar["exr"].pct_change(i).dropna(how="all"))[1] < threshold:
#         macvarst[f"exr{i}"] = macvar["exr"].pct_change(i)
#     if adfuller(macvar["exr"].pct_change(i).diff().dropna(how="all"))[1] < threshold:
#         macvarst[f"exr{i}_2"] = macvar["exr"].pct_change(i).diff()

#     if adfuller(macvar["cpi"].pct_change(i).dropna(how="all"))[1] < threshold:
#         macvarst[f"cpi{i}"] = macvar["cpi"].pct_change(i)
#     if adfuller(macvar["cpi"].pct_change(i).diff().dropna(how="all"))[1] < threshold:
#         macvarst[f"cpi{i}_2"] = macvar["cpi"].pct_change(i).diff()

#     if adfuller(macvar["gdp"].pct_change(i).dropna(how="all"))[1] < threshold:
#         macvarst[f"gdp{i}"] = macvar["gdp"].pct_change(i)
#     if adfuller(macvar["gdp"].pct_change(i).diff().dropna(how="all"))[1] < threshold:
#         macvarst[f"gdp{i}_2"] = macvar["gdp"].pct_change(i).diff()

#     if adfuller(macvar["rs"].pct_change(i).dropna(how="all"))[1] < threshold:
#         macvarst[f"rs{i}"] = macvar["rs"].pct_change(i)
#     if adfuller(macvar["rs"].pct_change(i).diff().dropna(how="all"))[1] < threshold:
#         macvarst[f"rs{i}_2"] = macvar["rs"].pct_change(i).diff()

#     if adfuller(macvar["pda"].pct_change(i).dropna(how="all"))[1] < threshold:
#         macvarst[f"pda{i}"] = macvar["pda"].pct_change(i)
#     if adfuller(macvar["pda"].pct_change(i).diff().dropna(how="all"))[1] < threshold:
#         macvarst[f"pda{i}_2"] = macvar["pda"].pct_change(i).diff()

In [ ]:
macvar = pd.DataFrame().assign(
    Description=xls7.loc[
        ("2021-06" > xls7["Description"]) & (xls7["Description"] > "1998-12")
    ]["Description"]
)
macvar.reset_index(drop=True, inplace=True)

macvar = pd.merge(
    macvar,
    xls1[["Description", "Cash Rate Target; monthly average"]],
    on="Description",
    how="left",
)
macvar = pd.merge(
    macvar,
    xls34[["Description", "AUD/USD Exchange Rate; see notes for further detail."]],
    on="Description",
    how="left",
)
macvar = pd.merge(
    macvar,
    xls5[["Description", "Consumer price index; All groups"]],
    on="Description",
    how="left",
)
macvar = pd.merge(
    macvar,
    xls6[["Description", "Gross domestic product (GDP); Chain volume"]],
    on="Description",
    how="left",
)
macvar = pd.merge(
    macvar,
    xls7[["Description", "Retail sales; All industries; Current price"]],
    on="Description",
    how="left",
)
macvar = pd.merge(
    macvar,
    xls7[["Description", "Private dwelling approvals"]],
    on="Description",
    how="left",
)

macvar = pd.merge(
    macvar,
    ASX200data[["Description", "Adj Close"]],
    on="Description",
    how="left",
)



In [ ]:
macvar = macvar.set_axis(
    ["date", "ir", "exr", "cpi", "gdp", "rs", "pda", "asx"], axis=1, inplace=False
)


macvar["ir"] = macvar["ir"].astype(float, errors="raise")
macvar["exr"] = macvar["exr"].astype(float, errors="raise")
macvar["cpi"] = macvar["cpi"].astype(float, errors="raise")
macvar["gdp"] = macvar["gdp"].astype(float, errors="raise")
macvar["rs"] = macvar["rs"].astype(float, errors="raise")
macvar["pda"] = macvar["pda"].astype(float, errors="raise")
macvar["asx"] = macvar["asx"].astype(float, errors="raise")
macvar = macvar.interpolate()

In [ ]:
macvar

In [ ]:
# A function used to obtain a stationay series from the original series
## Order: series itself; 1sr order; seasonal order; annual order; 2nd order. 
threshold = 0.01

def selectStationaySeries(variable_tar):
        if adfuller(variable_tar.dropna(how="all"))[1] < threshold:
            stationary_variable = variable_tar
            suffix =  "original"
        elif adfuller(variable_tar.pct_change(1).dropna(how="all"))[1] < threshold:
            stationary_variable = variable_tar.pct_change(1).dropna(how="all")
            suffix = "1storderdiff"
        elif adfuller(variable_tar.pct_change(3).dropna(how="all"))[1] < threshold:
            stationary_variable = variable_tar.pct_change(3).dropna(how="all")
            suffix = "seasonaldiff"
        elif adfuller(variable_tar.pct_change(12).dropna(how="all"))[1] < threshold:
            stationary_variable = variable_tar.pct_change(12).dropna(how="all")
            suffix = "annualdiff"     
        elif adfuller(variable_tar.pct_change(1).diff().dropna(how="all"))[1] < threshold:
            stationary_variable = variable_tar.pct_change(1).diff().dropna(how="all")
            suffix = "2ndorderdiff"
        else:
            print("not found")
        return(pd.DataFrame(stationary_variable))


In [ ]:
macvarst = macvar["date"] 
for vn in macvar.columns.values[1:]:
    macvarst = pd.concat([macvarst, selectStationaySeries(macvar[vn])], axis=1)
   


In [ ]:
# macvarst = macvar
# lag = [1]

# for i in lag:
#     if adfuller(macvar["ir"].pct_change(i).dropna(how="all"))[1] < threshold:
#         macvarst[f"ir{i}"] = macvar["ir"].pct_change(i)
#     if adfuller(macvar["ir"].pct_change(i).diff().dropna(how="all"))[1] < threshold:
#         macvarst[f"ir{i}_2"] = macvar["ir"].pct_change(i).diff()

#     if adfuller(macvar["exr"].pct_change(i).dropna(how="all"))[1] < threshold:
#         macvarst[f"exr{i}"] = macvar["exr"].pct_change(i)
#     if adfuller(macvar["exr"].pct_change(i).diff().dropna(how="all"))[1] < threshold:
#         macvarst[f"exr{i}_2"] = macvar["exr"].pct_change(i).diff()

#     if adfuller(macvar["cpi"].pct_change(i).dropna(how="all"))[1] < threshold:
#         macvarst[f"cpi{i}"] = macvar["cpi"].pct_change(i)
#     if adfuller(macvar["cpi"].pct_change(i).diff().dropna(how="all"))[1] < threshold:
#         macvarst[f"cpi{i}_2"] = macvar["cpi"].pct_change(i).diff()

#     if adfuller(macvar["gdp"].pct_change(i).dropna(how="all"))[1] < threshold:
#         macvarst[f"gdp{i}"] = macvar["gdp"].pct_change(i)
#     if adfuller(macvar["gdp"].pct_change(i).diff().dropna(how="all"))[1] < threshold:
#         macvarst[f"gdp{i}_2"] = macvar["gdp"].pct_change(i).diff()

#     if adfuller(macvar["rs"].pct_change(i).dropna(how="all"))[1] < threshold:
#         macvarst[f"rs{i}"] = macvar["rs"].pct_change(i)
#     if adfuller(macvar["rs"].pct_change(i).diff().dropna(how="all"))[1] < threshold:
#         macvarst[f"rs{i}_2"] = macvar["rs"].pct_change(i).diff()

#     if adfuller(macvar["pda"].pct_change(i).dropna(how="all"))[1] < threshold:
#         macvarst[f"pda{i}"] = macvar["pda"].pct_change(i)
#     if adfuller(macvar["pda"].pct_change(i).diff().dropna(how="all"))[1] < threshold:
#         macvarst[f"pda{i}_2"] = macvar["pda"].pct_change(i).diff()

In [ ]:
month = [int(my_str.split("-")[1]) for my_str in macvarst["date"].values]
quater = [(m - 1) // 3 + 1 for m in month]
monthdummies = pd.get_dummies(month, prefix="month")
quaterdummies = pd.get_dummies(quater, prefix="quater")
dummies = pd.concat([monthdummies, quaterdummies], axis=1)
#macvarst  =  pd.concat([macvarst,dummies],axis =1)
#macvarst = pd.concat([macvarst, monthdummies], axis=1)

In [ ]:
from sklearn.preprocessing import PolynomialFeatures
from sklearn.preprocessing import StandardScaler

# skip reading
# macvarsta = macvarst[
#     macvarst.columns.difference(["ir", "exr", "cpi", "gdp", "rs", "pda"])
# ]
# macvarsta = macvarst[macvarst.columns.difference(['ir','exr','cpi','gdp','rs','pda','loghpidiff'])]

macvarsta = macvarst  
matrixmac = macvarsta.corr().round(2)
macdata = macvarsta.loc[
    ("2019-12" > macvarsta["date"]) & (macvarsta["date"] > "2000-12")
]



In [ ]:
macvarsta

In [ ]:
pca = PCA()
X_train = macdata.loc[:, macdata.columns != "date"]
poly= PolynomialFeatures(degree=2)
scaler = StandardScaler()

#X_train_poly = poly.fit_transform(X_train)
#X_train_std = scaler.fit_transform(X_train_poly)
X_train_std = scaler.fit_transform(X_train)



In [ ]:
X_train 

In [ ]:
pca.fit(X_train_std)
cumsum = np.cumsum(pca.explained_variance_ratio_)
d = np.argmax(cumsum >= 0.9) + 1
print(d)

pca_d = PCA(n_components=d)
pca_d.fit(X_train)
X_pca_d = pca_d.transform(X_train)
PCnames = []
for i in range(1, d + 1, 1):
    PCnames.append(f"PC{i}")
pcadf = pd.DataFrame(X_pca_d, columns=PCnames)

In [ ]:
pca_avg = pd.concat(
    [
        pcadf,
        agg["mean"]
        .loc[("2019-12" > agg["Description"]) & (agg["Description"] > "2001-01")]
        .reset_index(drop=True),
    ],
    axis=1,
)
# skip reading
paneldata = []
lst = ["postcode", "date", "logHPI", "difflogHPI"]
for key in DataFrameDict_postcode.keys():
    df = pd.DataFrame([], columns=lst)
    df["date"] = DataFrameDict_postcode[key]["value_at_date"].loc[
        ("2019-12" > DataFrameDict_postcode[key]["value_at_date"])
        & (DataFrameDict_postcode[key]["value_at_date"] > "2001-01")
    ]
    df["logHPI"] = np.log2(
        DataFrameDict_postcode[key]["Hedonic Home Value Index"].loc[
            ("2019-12" > DataFrameDict_postcode[key]["value_at_date"])
            & (DataFrameDict_postcode[key]["value_at_date"] > "2000-12")
        ]
    )
    df["postcode"] = key
    df["difflogHPI"] = (
        (
            np.log2(
                DataFrameDict_postcode[key]["Hedonic Home Value Index"].loc[
                    ("2019-12" > DataFrameDict_postcode[key]["value_at_date"])
                    & (DataFrameDict_postcode[key]["value_at_date"] > "2000-12")
                ]
            )
        )
        .diff(1)
        .dropna()
    )
    data = pd.concat([df.reset_index(drop=True), pca_avg], axis=1)
    paneldata.append(data)
paneldf = pd.concat(paneldata, ignore_index=True)
paneldf["date"] = pd.to_datetime(paneldf["date"])
panelData = paneldf.set_index(["postcode", "date"])

In [ ]:
DataFrameDict_PC = {elem: pd.DataFrame for elem in UniquePostcode}
lst = ["postcode", "date", "logHPI", "difflogHPI"]
# Calling DataFrame constructor on list
for key in DataFrameDict_PC.keys():
    df = pd.DataFrame([], columns=lst)
    df["date"] = DataFrameDict_postcode[key]["value_at_date"].loc[
        ("2019-12" > DataFrameDict_postcode[key]["value_at_date"])
        & (DataFrameDict_postcode[key]["value_at_date"] > "2001-01")
    ]
    df["logHPI"] = np.log2(
        DataFrameDict_postcode[key]["Hedonic Home Value Index"].loc[
            ("2019-12" > DataFrameDict_postcode[key]["value_at_date"])
            & (DataFrameDict_postcode[key]["value_at_date"] > "2000-12")
        ]
    )
    df["postcode"] = key
    df["difflogHPI"] = (
        (
            np.log2(
                DataFrameDict_postcode[key]["Hedonic Home Value Index"].loc[
                    ("2019-12" > DataFrameDict_postcode[key]["value_at_date"])
                    & (DataFrameDict_postcode[key]["value_at_date"] > "2000-12")
                ]
            )
        )
        .diff(1)
        .dropna()
    )
    data = pd.concat([df.reset_index(drop=True), pca_avg], axis=1)
    DataFrameDict_PC[key] = data

In [ ]:
PCnames = []
for i in range(1, d + 1, 1):
    PCnames.append(f"PC{i}")
PCnames.append("mean")
df = pd.DataFrame(pca_avg, columns=PCnames)

nobs = 28  # post-covid data is used as the test data set
df_train, df_test = df[0:-nobs], df[-nobs:]
# Check size

start_point = df_train.shape[0]
print(df_train.shape)
print(df_test.shape)
model = VAR(df_train)
for i in [1,2,3, 4, 6, 7, 8]:
    result = model.fit(i)
    print("Lag Order =", i)
    print("AIC : ", result.aic)
    print("BIC : ", result.bic)
    print("FPE : ", result.fpe)
    print("HQIC: ", result.hqic, "\n")

In [ ]:
model_fitted = model.fit(2)
model_fitted.summary()
from statsmodels.stats.stattools import durbin_watson

out = durbin_watson(model_fitted.resid)


for col, val in zip(df.columns, out):
    print(col, ":", round(val, 2))

lag_order = model_fitted.k_ar
forecast_input = df_train.values[-lag_order:]

df_forecast = []
for i in range(1, nobs + 1, 1):
    forecast_input = df.values[
        -lag_order + start_point + i - 1 : start_point + i - 1,
    ]
    df_forecast.append(
        model_fitted.forecast(y=forecast_input, steps=1)[
            0,
        ][-1]
    )


df_results = pd.DataFrame(df_forecast, index=df.index[-nobs:], columns=["forecast"])

forecast_input_lt = df_train.values[-lag_order:]
fc = model_fitted.forecast(y=forecast_input_lt, steps=nobs)

df_results_lt = pd.DataFrame(
    fc, index=df.index[-nobs:], columns=df.columns + "forecast"
)

In [ ]:
df_forecast

In [ ]:
date_frame = pd.DataFrame(
    agg["Description"]
    .loc[("2019-12" > agg["Description"]) & (agg["Description"] > "2001-01")]
    .reset_index(drop=True)
)
x = date_frame[-nobs:]["Description"]
color1 = "#0085c3"
color2 = "#7ab800"
color3 = "#dc5034"
y = df_results["forecast"]
y2 = df_test["mean"][-nobs:]
y3 = savgol_filter(df_results_lt["meanforecast"], 5, 3)
fig = plt.figure(figsize=(20, 6))
ax = fig.add_subplot(111)
ax.plot(x, y2, marker="o", color=color1, label="Average HPI Growth Rate")
ax.plot(
    x, y, marker="o", ls="--", color=color2, label="Average HPI Growth Rate Forecast"
)
ax.plot(x, y3, ls="-.", color=color3, label="Average HPI Growth Rate Forecast (LT)")

ax.grid(ls=":", color="gray", alpha=0.6)

ax.legend(loc="upper left", fontsize=10)

plt.xticks(rotation=45, fontsize=12)
plt.yticks(fontsize=12)

# Mixed frequency data
- Introducing data at smaller scales with lower frequencies (e.g. annual) 
- Finding the principal components -> new risk factors at smaller scales
- New VAR models: average growth rate at smaller scales (annual)
- Combining average growth rates from VAR models
- Using risk factors(OLS) to explain excess house price growth rate
- Downsampling macroeconomic data (monthly) to annual date to build VAR model
- Deleting new risk factors when multicollinearity arises (?) 

In [ ]:
import re


def nan_dum(df):
    df1_ohe = df
    nan_df = df1_ohe.loc[:, df1_ohe.columns.str.endswith("_nan")]
    pattern = "^([^_]*)_"
    regex = re.compile(pattern)
    for index in df1_ohe.index:
        for col_nan in nan_df.columns:
            if df1_ohe.loc[index, col_nan] == 1:
                col_id = regex.search(col_nan).group(1)
                targets = df1_ohe.columns[df1_ohe.columns.str.startswith(col_id + "_")]
                df1_ohe.loc[index, targets] = np.nan
    df1_ohe.drop(
        df1_ohe.columns[df1_ohe.columns.str.endswith("_nan")], axis=1, inplace=True
    )
    return df1_ohe

In [ ]:
def person_cat(postcode, lst):
    df_tmp = pd.DataFrame([], columns=lst)
    for key in DataFrameDict_HILDA.keys():
        lst0 = list(set(lst) - set(DataFrameDict_HILDAp[key].columns))
        lst1 = list(set(lst) - set(lst0))
        df = (
            DataFrameDict_HILDAp[key][lst1]
            .loc[DataFrameDict_HILDAp[key]["hhpcode"] == postcode]
            .astype(str)
        )
        df["date"] = str(key)
        df_tmp = pd.concat([df_tmp, df], axis=0, ignore_index=True)
    df_dummy = pd.get_dummies(df_tmp, columns=df_tmp.columns[:-1], dummy_na=True)
    dfHILDA = df_dummy.groupby("date").mean().reset_index()
    return nan_dum(dfHILDA)

In [ ]:
def house_cat(postcode, lst):
    df_tmp = pd.DataFrame([], columns=lst)
    for key in DataFrameDict_HILDA.keys():
        lst0 = list(set(lst) - set(DataFrameDict_HILDA[key].columns))
        lst1 = list(set(lst) - set(lst0))
        df = (
            DataFrameDict_HILDA[key][lst1]
            .loc[DataFrameDict_HILDA[key]["hhpcode"] == postcode]
            .astype(str)
        )
        df["date"] = str(key)
        df_tmp = pd.concat([df_tmp, df], axis=0, ignore_index=True)
    df_dummy = pd.get_dummies(df_tmp, columns=df_tmp.columns[:-1], dummy_na=True)
    dfHILDA = df_dummy.groupby("date").mean().reset_index()
    return nan_dum(dfHILDA)

In [ ]:
def house_cat(postcode, lst):
    df_tmp = pd.DataFrame([], columns=lst)
    for key in DataFrameDict_HILDA.keys():
        lst0 = list(set(lst) - set(DataFrameDict_HILDA[key].columns))
        lst1 = list(set(lst) - set(lst0))
        df = (
            DataFrameDict_HILDA[key][lst1]
            .loc[DataFrameDict_HILDA[key]["hhpcode"] == postcode]
            .astype(str)
        )
        df["date"] = str(key)
        df_tmp = pd.concat([df_tmp, df], axis=0, ignore_index=True)
    dfHILDA = df_tmp.groupby("date").mean().reset_index()
    return nan_dum(dfHILDA)

In [ ]:
def person_num(postcode, lst):
    df_tmp = pd.DataFrame([], columns=lst)
    for key in DataFrameDict_HILDA.keys():
        lst0 = list(set(lst) - set(DataFrameDict_HILDAp[key].columns))
        lst1 = list(set(lst) - set(lst0))
        df = DataFrameDict_HILDAp[key][lst1].loc[
            DataFrameDict_HILDAp[key]["hhpcode"] == postcode
        ]
        df["date"] = str(key)
        df_tmp = pd.concat([df_tmp, df], axis=0, ignore_index=True)
    dfHILDA = df_tmp.groupby("date").agg({lambda x: x.mean(skipna=False)})
    dfHILDA.columns = lst
    return dfHILDA.reset_index()

In [ ]:
def house_num(postcode, lst):
    df_tmp = pd.DataFrame([], columns=lst)
    for key in DataFrameDict_HILDA.keys():
        lst0 = list(set(lst) - set(DataFrameDict_HILDA[key].columns))
        lst1 = list(set(lst) - set(lst0))
        df = DataFrameDict_HILDA[key][lst1].loc[
            DataFrameDict_HILDA[key]["hhpcode"] == postcode
        ]
        df["date"] = str(key)
        df_tmp = pd.concat([df_tmp, df], axis=0, ignore_index=True)
    dfHILDA = df_tmp.groupby("date").agg({lambda x: x.mean(skipna=False)})
    dfHILDA.columns = lst
    return dfHILDA.reset_index()

In [ ]:
# lst1 variables in HOUSE files
# lst2 variables in PERSON files
def HILDAext_cat(postcode, lst1, lst2):
    dfHILDA1 = house_cat(postcode, lst1)
    dfHILDA1["date"] = pd.to_datetime(dfHILDA1["date"]) + pd.offsets.YearEnd()
    dfHILDA2 = person_cat(postcode, lst2)
    dfHILDA2["date"] = pd.to_datetime(dfHILDA2["date"]) + pd.offsets.YearEnd()
    dfHILDA = pd.merge(dfHILDA1, dfHILDA2, on="date", how="left")
    return dfHILDA

In [ ]:
def HILDAext_num(postcode, lst1, lst2):
    dfHILDA1 = house_num(postcode, lst1)
    dfHILDA1["date"] = pd.to_datetime(dfHILDA1["date"]) + pd.offsets.YearEnd()
    dfHILDA2 = person_num(postcode, lst2)
    dfHILDA2["date"] = pd.to_datetime(dfHILDA2["date"]) + pd.offsets.YearEnd()
    dfHILDA = pd.merge(dfHILDA1, dfHILDA2, on="date", how="left")
    return dfHILDA

In [ ]:
def q1(x):
    return x.quantile(0.25)
def q3(x):
    return x.quantile(0.75)

In [ ]:
lst_age = [f'hgdob{i}' for i in range(1, 21)]

def HILDAext_age(postcode):    
    df_tmp = pd.DataFrame()
    for key in DataFrameDict_HILDA.keys():
        df = DataFrameDict_HILDA[key][lst_age].loc[DataFrameDict_HILDA[key]["hhpcode"] == postcode]
        df["date"] = str(key)
        df = df.melt(id_vars='date', var_name='Indicator name', value_name='birth_date').sort_values('date', ascending=False).reset_index(drop=True)
        df['birthDate'] = pd.to_datetime(df['birth_date'].replace(r'-','na',regex=True),errors = 'coerce')
        df['year'] = pd.DatetimeIndex(df['birthDate']).year
        df['age'] = key +1 - df['year']
        df_tmp = pd.concat([df_tmp, df[['date','age']]], axis=0, ignore_index=True)
    f = {'age': ['median', 'std', q1, q3,'count']} 
    dfHILDAage = df_tmp.groupby('date').agg(f)
    dfHILDAage = dfHILDAage.set_axis(dfHILDAage.columns.map(''.join), axis=1, inplace=False).reset_index()
    dfHILDAage["date"] = pd.to_datetime(dfHILDAage["date"]) + pd.offsets.YearEnd()
    return dfHILDAage

In [ ]:
#HILDAext_age(postcode)
#HILDAext_age1(postcode)
#which shows the household and enumerated dataset are different sources of data

# Variables from HILDA

[ancob] History: Country of birth

[edhigh1] History: Highest education level achieved

[anatsi]	History: Aboriginal or Torres Strait Islander origin


[chkb12]	Check B12 employment status

[wsce] DV: Current weekly gross wages & salary, all jobs, includes estimated from net ($) [weighted topcode]

[baynoa] Total amount held in these accounts combined [weighted topcode]

[xpgroci]	DV: Household weekly expenditure on all groceries [imputed]

[hsdebt]	DV: Total Home Debt ($) [weighted topcode]

[hgdob1] to [hgdob20] Date of birth

[hgsex]	Sex


```[hgyob]	Year of Birth```



In [ ]:
varnames_catp = ["ancob", "edhigh1", "anatsi", "chkb12", "sex"]
# varnames_nump = ["wsce", "baynoa"]
varnames_nump = ["wsce"]
varnames_cath = []
#varnames_cath = ["xpgroci"]
varnames_numh = ["hsdebt"]

# Example
In this example, postcode is fixed to be "2018". We can see the results of two data extraction functions from HILDA data.  

In [ ]:
HILDAext_num("2018", varnames_numh, varnames_nump)

In [ ]:
HILDAext_cat("2018", varnames_cath, varnames_catp)

In [ ]:
# import statsmodels.api as sm

# Residuals_PC = {elem: pd.DataFrame for elem in UniquePostcode}
# lst = ["postcode", "date", "residuals"]
# for key in Residuals_PC.keys():
#     df = pd.DataFrame([], columns=lst)
#     df["date"] = DataFrameDict_PC[key].iloc[:, 1]
#     df["date"] = pd.to_datetime(df["date"]) + pd.offsets.MonthEnd()
#     df["postcode"] = key
#     X = DataFrameDict_PC[key].iloc[:, 4:].values
#     Y = DataFrameDict_PC[key].iloc[:, 3].values
#     X = sm.add_constant(X)
#     result = sm.OLS(Y, X).fit()
#     df["residuals"] = result.resid
#     data = pd.concat([df.reset_index(drop=True), pca_avg], axis=1)
#     HILDAextract_num = HILDAext_num(key, varnames_numh, varnames_nump)
#     HILDAextract_cat = HILDAext_cat(key, varnames_cath, varnames_catp)
#     HILDAextract = pd.merge(HILDAextract_num, HILDAextract_cat, on="date", how="left")
#     Residuals_PC[key] = pd.merge(data, HILDAextract, on="date", how="left")
#     print(key + ": success")

In [ ]:
# Residuals_PC['3150']

In [ ]:
# def HILDA2PC(key):
#     df = Residuals_PC[key].interpolate("bfill").interpolate("ffill")
#     df = df.set_index("date")
#     df1 = df.resample("Y").mean().iloc[:, d + 2 :]
#     x = df1.loc[:, :].values
#     x = StandardScaler().fit_transform(x)
#     pca = PCA()
#     pca.fit(x)
#     cumsum = np.cumsum(pca.explained_variance_ratio_)
#     d0 = np.argmax(cumsum >= 0.9) + 1
#     pca_d = PCA(n_components=d0)
#     pca_d.fit(x)
#     X_pca_d = pca_d.transform(x)
#     PCnames = []
#     for i in range(1, d0 + 1, 1):
#         PCnames.append(f"l_PC{i}")
#     pcadf = pd.DataFrame(X_pca_d, columns=PCnames)
#     return pcadf

In [ ]:
# dti = pd.date_range("2001-12-31", periods=19, freq="Y")


# def HILDApcavg(key):
#     df = Residuals_PC[key].interpolate("bfill").interpolate("ffill")
#     df1 = df.set_index("date")
#     df2 = df1.resample("Y").mean().iloc[:, :]
#     df3 = pd.concat([df2.iloc[:, 0].reset_index(drop=True), HILDA2PC(key)], axis=1)
#     return df3.set_index(dti).resample("M").mean().interpolate(method="linear")

In [ ]:
# l_DataFrameDict_PC = {elem: pd.DataFrame for elem in UniquePostcode}
# lst = ["postcode", "date", "difflogHPI"]
# # Calling DataFrame constructor on list
# for key in l_DataFrameDict_PC.keys():
#     df = pd.DataFrame([], columns=lst)
#     df["date"] = DataFrameDict_postcode[key]["value_at_date"].loc[
#         ("2019-12" > DataFrameDict_postcode[key]["value_at_date"])
#         & (DataFrameDict_postcode[key]["value_at_date"] > "2001-01")
#     ]
#     df["postcode"] = key
#     df["difflogHPI"] = (
#         (
#             np.log2(
#                 DataFrameDict_postcode[key]["Hedonic Home Value Index"].loc[
#                     ("2019-12" > DataFrameDict_postcode[key]["value_at_date"])
#                     & (DataFrameDict_postcode[key]["value_at_date"] > "2000-12")
#                 ]
#             )
#         )
#         .diff(1)
#         .dropna()
#     )
#     data = pd.concat([df.reset_index(drop=True), pca_avg], axis=1)
#     data["date"] = pd.to_datetime(data["date"]) + pd.offsets.MonthEnd()
#     data = data.set_index("date")
#     dframe = pd.concat([data, HILDApcavg(key)], axis=1)
#     l_DataFrameDict_PC[key] = dframe.interpolate("bfill")

In [ ]:
# l_DataFrameDict_PC["2018"].info()

# Conclusion 
- This is a 2-level model: more data should be included 
- Average R-squared increases from 37% to 50% with the introduction of HILDA data.
- Next step: cutting down number of parameters 

In [ ]:
# # New average R-squared after the introduction of HILDA data
# import statsmodels.api as sm

# Rsq = []
# for key in l_DataFrameDict_PC.keys():
#     X = l_DataFrameDict_PC[key].iloc[:, 2:].values
#     Y = l_DataFrameDict_PC[key].iloc[:, 1].values
#     X = sm.add_constant(X)
#     result = sm.OLS(Y, X).fit()
#     Rsq.append(result.rsquared)
# from statistics import mean

# cleanedRsq = [x for x in Rsq if str(x) != "nan"]
# mean(cleanedRsq)

In [ ]:
aveHPI_lv3 = []
lst = ["postcode", "sa3", "state", "logHPI", "logHPIdiff"]
# Calling DataFrame constructor on list
for key in DataFrameDict_postcode.keys():
    df = pd.DataFrame([], columns=lst)
    df["Description"] = DataFrameDict_postcode[key]["value_at_date"].loc[
        ("2019-12" > DataFrameDict_postcode[key]["value_at_date"])
        & (DataFrameDict_postcode[key]["value_at_date"] > "1999-12")
    ]
    df["logHPI"] = np.log2(
        DataFrameDict_postcode[key]["Hedonic Home Value Index"].loc[
            ("2019-12" > DataFrameDict_postcode[key]["value_at_date"])
            & (DataFrameDict_postcode[key]["value_at_date"] > "1999-12")
        ]
    )
    df["postcode"] = key
    df["sa3"] = DataFrameDict_postcode[key]["sa3_name16"].loc[
        ( DataFrameDict_postcode[key]["postcode"] == key)]
    df["state"] = DataFrameDict_postcode[key]["state"].loc[
        ( DataFrameDict_postcode[key]["postcode"] == key)]
    
    df["logHPIdiff"] = df["logHPI"].diff(1)
    aveHPI_lv3.append(df)

aveHPIdf_lv3 = pd.concat(aveHPI_lv3)
aveHPIdf_lv3 = aveHPIdf_lv3.reset_index(drop=True)
aveHPIdf_lv3['date'] = pd.to_datetime(aveHPIdf_lv3['Description'],errors = 'coerce')
aveHPIdf_lv3['year'] = pd.DatetimeIndex(aveHPIdf_lv3['date']).year

In [ ]:
aveHPIdf_lv3

In [ ]:
avg_1_y = aveHPIdf_lv3.groupby(["year"])["logHPIdiff"].agg(["mean"]).reset_index()
avg_1_y['date'] = pd.to_datetime(avg_1_y['year'].apply(str)) + pd.offsets.YearEnd()
avg_1_y= avg_1_y.set_index("date")

avg_1 = aveHPIdf_lv3.groupby(["date"])["logHPIdiff"].agg(["mean"]).reset_index()
avg_1= avg_1.set_index("date")

avg_2 = aveHPIdf_lv3.groupby(["state","year"])["logHPIdiff"].agg(["mean"]).reset_index()
avg_2['date'] = pd.to_datetime(avg_2['year'].apply(str)) + pd.offsets.YearEnd()
avg_2= avg_2.set_index("date")


avg_3 = aveHPIdf_lv3.groupby(["sa3","year"])["logHPIdiff"].agg(["mean"]).reset_index()
avg_3['date'] = pd.to_datetime(avg_3['year'].apply(str)) + pd.offsets.YearEnd()
avg_3= avg_3.set_index("date")




In [ ]:
# import statsmodels.api as sm

# DataFrameDict_CL = {elem: pd.DataFrame for elem in UniquePostcode}
# lst = ["postcode","sa3" ,"state" , "date"]
# for key in DataFrameDict_CL.keys():
#     df = pd.DataFrame([], columns=lst)
#     df["date"] = DataFrameDict_postcode[key]["value_at_date"].loc[("2019-12" > DataFrameDict_postcode[key]["value_at_date"])
#         & (DataFrameDict_postcode[key]["value_at_date"] > "1999-12")]
#     df["date"] =  pd.to_datetime(df["date"])
#     df["difflogHPI"] = (
#         (
#             np.log2(
#                 DataFrameDict_postcode[key]["Hedonic Home Value Index"].loc[
#                     ("2019-12" > DataFrameDict_postcode[key]["value_at_date"])
#                     & (DataFrameDict_postcode[key]["value_at_date"] > "2000-12")
#                 ]
#             )
#         )
#         .diff(1)
#         .dropna()
#     )
#     df["postcode"] = key
#     sa3name =  DataFrameDict_postcode[key]["sa3_name16"].loc[( DataFrameDict_postcode[key]["postcode"] == key)].unique()[0]
#     statename = DataFrameDict_postcode[key]["state"].loc[( DataFrameDict_postcode[key]["postcode"] == key)].unique()[0]
#     df= df.set_index("date")
#     df0 = pd.concat([df,avg_1['mean']], axis=1,join = "inner")
#     df1 = pd.concat([df0,avg_2['mean'].loc[avg_2['state'] == statename]], axis=1)
#     df2 = pd.concat([df1,avg_3['mean'].loc[avg_3['sa3'] == sa3name]], axis=1)
#     df_re = df2.drop(columns=df2.columns[[0,1,2]]).drop(df2.index[range(12)], axis=0)
#     df_out = pd.DataFrame([])
#     df_out['HPI'] = df_re['difflogHPI']
#     df_out['l0'] = df_re.iloc[:,1]
#     df_out['l1'] = df_re.iloc[:,2].interpolate() - df_re.iloc[:,1].interpolate() 
#     df_out['l2'] = df_re.iloc[:,3].interpolate() - df_re.iloc[:,2].interpolate()
#     df_out['l3'] = df_out['HPI']- df_out['l0']- df_out['l1'] - df_out['l2']
#     DataFrameDict_CL[key] = df_out.drop(df2.index[[-1,12]], axis=0)
    
    

In [ ]:
import statsmodels.api as sm

DataFrameDict_CL = {elem: pd.DataFrame for elem in UniquePostcode}
lst = ["postcode","sa3" ,"state" , "date"]
for key in DataFrameDict_CL.keys():
    df = pd.DataFrame([], columns=lst)
    df["date"] = DataFrameDict_postcode[key]["value_at_date"].loc[("2019-12" > DataFrameDict_postcode[key]["value_at_date"])
        & (DataFrameDict_postcode[key]["value_at_date"] > "1999-12")]
    df["date"] =  pd.to_datetime(df["date"])
    df["difflogHPI"] = (
        (
            np.log2(
                DataFrameDict_postcode[key]["Hedonic Home Value Index"].loc[
                    ("2019-12" > DataFrameDict_postcode[key]["value_at_date"])
                    & (DataFrameDict_postcode[key]["value_at_date"] > "2000-12")
                ]
            )
        )
        .diff(1)
        .dropna()
    )
    df["postcode"] = key
    sa3name =  DataFrameDict_postcode[key]["sa3_name16"].loc[( DataFrameDict_postcode[key]["postcode"] == key)].unique()[0]
    statename = DataFrameDict_postcode[key]["state"].loc[( DataFrameDict_postcode[key]["postcode"] == key)].unique()[0]
    df= df.set_index("date")
    df0 = pd.concat([df,avg_1['mean']], axis=1,join = "inner")
    df1 = pd.concat([df0,avg_2['mean'].loc[avg_2['state'] == statename]], axis=1)
    df2 = pd.concat([df1,avg_3['mean'].loc[avg_3['sa3'] == sa3name]], axis=1)
    df_re = df2.drop(columns=df2.columns[[0,1,2]]).drop(df2.index[range(12)], axis=0)
    df_out = pd.DataFrame([])
    df_out['HPI'] = df_re['difflogHPI']
    df_out['l0'] = df_re.iloc[:,1]
    df_out['l1'] = df_re.iloc[:,2].interpolate() - df_re.iloc[:,1].interpolate()
    df_out['l2'] = df_re.iloc[:,3].interpolate() - df_re.iloc[:,2].interpolate()
    df_out['l3'] = df_out['HPI']- df_out['l0']- df_out['l1'] - df_out['l2'] 
    DataFrameDict_CL[key] = df_out.drop(df2.index[[-1,12]], axis=0)

In [ ]:
DataFrameDict_CL['2018']

In [ ]:
# # New average R-squared after the introduction of HILDA data
# import statsmodels.api as sm

# Rsq = []
# for key in DataFrameDict_CL.keys():
#     X = l_DataFrameDict_PC[key].iloc[:, 2:].values
#     Y = DataFrameDict_CL[key].iloc[:,4].values
#     X = sm.add_constant(X)
#     result = sm.OLS(Y, X).fit()
#     Rsq.append(result.rsquared)
# from statistics import mean

# cleanedRsq = [x for x in Rsq if str(x) != "nan"]
# mean(cleanedRsq)

In [ ]:
UniqueNames_Hsa3 = avg_3.sa3.unique()[:-1]
DataFrameDict_Hsa3 = {elem: pd.DataFrame for elem in UniqueNames_Hsa3}

In [ ]:
DataFrameDict_Hsa3.keys()

In [ ]:
for key in DataFrameDict_Hsa3.keys():
    lstpcCL= ndatahouses['postcode'].loc[ndatahouses['sa3_name16'] == key].unique()
    lst0 = list(set(lstpcCL) - set(HILDAallpc))
    lstpc  = list(set(lstpcCL) - set(lst0))
    df_tmp = pd.DataFrame([])
    for key_pc in lstpc:
        HILDAextract_num = HILDAext_num(key_pc, varnames_numh, varnames_nump)
        HILDAextract_cat = HILDAext_cat(key_pc, varnames_cath, varnames_catp)
        HILDAextract_age = HILDAext_age(key_pc)
        HILDAextract = pd.merge(HILDAextract_num, HILDAextract_cat, on="date", how="left")
        HILDAextract = pd.merge( HILDAextract , HILDAextract_age, on="date", how="left")
        df_tmp = pd.concat([df_tmp, HILDAextract], axis=0, ignore_index=True)
    dfHILDA = df_tmp.groupby("date").agg({lambda x: x.mean(skipna=True)})
    list1, list2 = zip(*dfHILDA.columns)
    dfHILDA.columns = list1
    DataFrameDict_Hsa3[key] = dfHILDA.reset_index()    
        


In [ ]:
with open('dict_of_dfs.pickle', 'wb') as f:
    pickle.dump(DataFrameDict_Hsa3, f)

In [ ]:
# with open('dict_of_dfs.pickle', 'rb') as f:
#     Test_Dict =  pickle.load(f)

In [ ]:
def HILDA2PCsa3(key):
    df = DataFrameDict_Hsa3[key].interpolate("bfill").interpolate("ffill")
    df = df.set_index("date")
    df1 = df.resample("Y").mean().iloc[:, :]
    x = df1.loc[:, :].values
    
    vrst = macvar["date"] 
    for vn in macvar.columns.values[1:]:
        macvarst = pd.concat([macvarst, selectStationaySeries(macvar[vn])], axis=1)
   
    
    x = StandardScaler().fit_transform(x)
    pca = PCA()
    pca.fit(x)
    cumsum = np.cumsum(pca.explained_variance_ratio_)
    d0 = np.argmax(cumsum >= 0.9) + 1
    pca_d = PCA(n_components=d0)
    pca_d.fit(x)
    X_pca_d = pca_d.transform(x)
    PCnames = []
    for i in range(1, d0 + 1, 1):
        PCnames.append(f"l_PC{i}")
    pcadf = pd.DataFrame(X_pca_d, columns=PCnames)
    return pcadf

In [ ]:
def selectStationaySeries2(variable_tar):
    count1 = np.isinf(variable_tar.pct_change(1)).values.sum()
    count2 = np.isinf(variable_tar.pct_change(1)).values.sum()
    
    if adfuller(variable_tar.interpolate("bfill").interpolate("ffill"))[1] < threshold:
            stationary_variable = variable_tar.interpolate("bfill").interpolate("ffill")
            suffix =  "original"
            print(suffix)
            return(pd.DataFrame(stationary_variable))
    
    
    elif count1 == 0:
        try:
            if adfuller(variable_tar.pct_change(1).interpolate("bfill").interpolate("ffill"))[1] < threshold:
                stationary_variable = variable_tar.pct_change(1).interpolate("bfill").interpolate("ffill")
                suffix = "1storderdiff"
                print(suffix)
                return(pd.DataFrame(stationary_variable))
        except np.linalg.LinAlgError as e1:
            print('except:', e1)
    
    
    elif count2 == 0:
        try: 
            if adfuller(variable_tar.pct_change(1).diff().dropna(how="all"))[1] < threshold:
                stationary_variable = variable_tar.pct_change(1).diff().dropna(how="all")
                suffix = "2ndorderdiff"
                print(suffix)
                return(pd.DataFrame(stationary_variable))
        except np.linalg.LinAlgError as e1:
            print('except:', e1)     
    
    else:
        print('not found')
        

             
       
        

In [ ]:
df = DataFrameDict_Hsa3[key].interpolate("bfill").interpolate("ffill")
df = df.set_index("date")
df1 = df.resample("Y").mean().iloc[:, :]
x = df1.loc[:, :].values
vrst = pd.DataFrame([])
for vn in df1.columns.values[1:]:
    vrst= pd.concat([vrst, selectStationaySeries2(df1[vn])], axis=1)
    

In [ ]:
vrst